In [1]:
# ================================================
# Project Alpha - Dataset Generation
# Notebook: 01_generate_dataset.ipynb
# Description: Generate realistic banking dataset with 100,000 customers
# Author: Shiva Sai Pulluri & Team
# ================================================

import numpy as np
import pandas as pd
import random
from faker import Faker

fake = Faker()

# Total customers
num_customers = 100000

churn_rates_by_age = {
    "18-29": 0.06,
    "30-39": 0.054,
    "40-49": 0.079,
    "50-59": 0.05,
    "60+": 0.04
}

churn_rates_by_marital_dependents = {
    "Single": 0.12,
    "Married": 0.08,
    "Divorced_0_dependents": 0.173,
    "Divorced_1+_dependents": 0.10
}

churn_rates_by_product = {
    "Savings": 0.08,
    "Current": 0.10,
    "CreditCard": 0.25,
    "Loan": 0.15,
    "High-Yield": 0.05
}

churn_rate_unresolved_complaints = 0.30
churn_rate_low_engagement = 0.20

customers = []

for i in range(num_customers):
    customer_id = fake.uuid4()
    age = np.random.randint(18, 75)
    gender = random.choice(['Male', 'Female', 'Other'])
    income_group = random.choice(['Low', 'Middle', 'High'])
    customer_type = random.choice(['Income Generating', 'Self-Employed', 'Student', 'Dependent'])
    residency = random.choice(['National Resident', 'Resident Alien', 'Non-Resident'])

    tenure = np.random.randint(1, 20)
    account_type = random.choice(['Savings', 'Current', 'CreditCard', 'Loan', 'High-Yield'])
    avg_balance = np.random.randint(500, 100000)

    deposits = np.random.randint(1000, 50000)
    withdrawals = np.random.randint(500, deposits)
    transactions_per_month = np.random.randint(5, 50)

    active_loans = np.random.choice([0, 1], p=[0.8, 0.2])
    loan_emi = np.random.randint(0, 5000) if active_loans else 0
    has_other_bank_loans = np.random.choice([0, 1], p=[0.6, 0.4])
    has_other_liabilities = np.random.choice([0, 1], p=[0.7, 0.3])

    products_with_bank = np.random.randint(1, 5)
    inquired_new_product = np.random.choice([0, 1], p=[0.5, 0.5])
    reported_income_increase = np.random.choice([0, 1], p=[0.4, 0.6])
    reported_dissatisfaction = np.random.choice([0, 1], p=[0.7, 0.3])
    issue_resolved = np.random.choice([0, 1]) if reported_dissatisfaction else None

    credit_score = random.choice(['Low', 'Medium', 'High'])
    overdraft_usage = np.random.randint(0, 5)

    has_external_insurance = np.random.choice([0, 1], p=[0.5, 0.5])
    defaulted_external_insurance = 1 if has_external_insurance and random.random() < 0.1 else 0

    loyalty_score = (tenure + products_with_bank) / 10

    age_segment = "18-29" if age < 30 else "30-39" if age < 40 else "40-49" if age < 50 else "50-59" if age < 60 else "60+"
    marital_status = random.choice(['Single', 'Married', 'Divorced'])
    dependents = np.random.randint(0, 4)
    marital_key = f"{marital_status}_{dependents}_dependents" if marital_status == 'Divorced' else marital_status

    churn_prob = churn_rates_by_age[age_segment]
    churn_prob += churn_rates_by_marital_dependents.get(marital_key, 0.1)
    churn_prob += churn_rates_by_product[account_type]

    if reported_dissatisfaction and not issue_resolved:
        churn_prob += churn_rate_unresolved_complaints

    if transactions_per_month < 10:
        churn_prob += churn_rate_low_engagement

    churn_prob = min(max(churn_prob, 0), 1)
    churn = 1 if random.random() < churn_prob else 0

    customers.append([
        customer_id, age, gender, income_group, customer_type, residency,
        tenure, account_type, avg_balance, transactions_per_month,
        deposits, withdrawals, active_loans, loan_emi, has_other_bank_loans, has_other_liabilities,
        products_with_bank, inquired_new_product, reported_income_increase, reported_dissatisfaction, issue_resolved,
        credit_score, overdraft_usage, has_external_insurance, defaulted_external_insurance,
        loyalty_score
    ])

columns = [
    'Customer_ID', 'Age', 'Gender', 'Income_Group', 'Customer_Type', 'Residency_Status',
    'Account_Tenure', 'Account_Type', 'Monthly_Avg_Balance', 'Account_Activity',
    'Monthly_Deposits', 'Monthly_Withdrawals', 'Active_Loans', 'Loan_EMI', 'Other_Bank_Loans', 'Other_Liabilities',
    'Products_with_Bank', 'Inquired_New_Product', 'Reported_Income_Increase', 'Reported_Dissatisfaction', 'Issue_Resolved',
    'Credit_Score', 'Overdraft_Usage', 'Has_External_Insurance', 'Defaulted_External_Insurance',
    'Loyalty_Score'
]

df_features = pd.DataFrame(customers, columns=columns)

# Save features (without churn)
df_features.to_csv('../data/features.csv', index=False)

# Create separate churn file
churn_df = pd.DataFrame({
    'Customer_ID': [customer[0] for customer in customers],
    'Churn': [1 if random.random() < churn_prob else 0 for customer in customers]
})

churn_df.to_csv('../data/hidden_churn_labels.csv', index=False)

print("✅ Dataset generation complete — features.csv and hidden_churn_labels.csv saved.")


✅ Dataset generation complete — features.csv and hidden_churn_labels.csv saved.


In [2]:
# 📌 Step: Add Behavioral Features

import pandas as pd
import numpy as np

# ✅ Load dataset with feature interactions
df = pd.read_csv("../data/features_with_interactions.csv")

# ✅ Feature 1: Product Stickiness
df["Product_Stickiness"] = df["Products_with_Bank"] * df["Account_Tenure"]

# ✅ Feature 2: Risk Profile Score
df["Risk_Profile_Score"] = (
    df["Overdraft_Usage"] * 0.4 +
    df["Loan_EMI"] * 0.3 +
    df["Credit_Score"].replace({'Low': 3, 'Medium': 2, 'High': 1}) * 0.3
)

# ✅ Feature 3: Complaint Risk Index
df["Complaint_Risk"] = df["Unresolved_Complaints"] * df["Reported_Dissatisfaction"]

# ✅ Feature 4: Avg Balance per Product
df["Avg_Balance_Per_Product"] = df["Monthly_Avg_Balance"] / (df["Products_with_Bank"] + 1)

# ✅ Feature 5: Deposit Dependency
df["Deposit_Dependency"] = df["Monthly_Deposits"] / (df["Monthly_Avg_Balance"] + 1)

# ✅ Feature 6: Engagement Momentum
df["Engagement_Momentum"] = df["Loyalty_Score"] * (1 + df["Inquired_New_Product"])

# ✅ Feature 7: Income Stability Proxy
df["Income_Stability_Proxy"] = df["Reported_Income_Increase"] / (df["Other_Liabilities"] + 1)

# ✅ Save final behavioral dataset
df.to_csv("../data/features_with_behavioral.csv", index=False)

print("✅ Behavioral features successfully added and dataset saved.")


C:\Users\pullu\AppData\Local\Temp\ipykernel_50668\3039070811.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Credit_Score"].replace({'Low': 3, 'Medium': 2, 'High': 1}) * 0.3


✅ Behavioral features successfully added and dataset saved.


In [3]:
import pandas as pd

# Load the behavioral dataset
df = pd.read_csv("../data/features_with_behavioral.csv")

# Ensure required columns are present (this avoids KeyErrors)
required_columns = [
    'Monthly_Withdrawals', 'Monthly_Deposits', 'Monthly_Avg_Balance',
    'Account_Activity', 'Account_Tenure', 'Overdraft_Usage', 'Loyalty_Score',
    'Loan_EMI', 'Reported_Dissatisfaction', 'Unresolved_Complaints',
    'Reported_Income_Increase', 'Products_with_Bank', 'Risk_Profile_Score',
    'Deposit_Dependency', 'Product_Stickiness', 'Engagement_Momentum',
    'Avg_Balance_Per_Product', 'Customer_Type', 'Age', 'Income_Group'
]
missing = [col for col in required_columns if col not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Define churn risk segments (personas)
df["Segment_HighWithdrawals_LowDeposits"] = (
    (df["Monthly_Withdrawals"] > df["Monthly_Deposits"] * 1.5)
).astype(int)

df["Segment_LowEngagement_HighBalance"] = (
    (df["Account_Activity"] < 0.3) & (df["Monthly_Avg_Balance"] > df["Monthly_Avg_Balance"].quantile(0.75))
).astype(int)

df["Segment_FrequentComplaints_Unresolved"] = (
    (df["Unresolved_Complaints"] == 1) & (df["Reported_Dissatisfaction"] == 1)
).astype(int)

df["Segment_YoungLowBalance"] = (
    (df["Age"] < 25) & (df["Monthly_Avg_Balance"] < df["Monthly_Avg_Balance"].quantile(0.25))
).astype(int)

df["Segment_HighEMI_LowTenure"] = (
    (df["Loan_EMI"] > df["Loan_EMI"].quantile(0.75)) & (df["Account_Tenure"] < 1)
).astype(int)

df["Segment_HighOverdraft_HighRisk"] = (
    (df["Overdraft_Usage"] > 0.5) & (df["Risk_Profile_Score"] > df["Risk_Profile_Score"].mean())
).astype(int)

df["Segment_HighBalance_NoIncomeIncrease"] = (
    (df["Monthly_Avg_Balance"] > df["Monthly_Avg_Balance"].quantile(0.75)) & (df["Reported_Income_Increase"] == 0)
).astype(int)

df["Segment_LowLoyalty_HighActivity"] = (
    (df["Loyalty_Score"] < df["Loyalty_Score"].quantile(0.25)) & (df["Account_Activity"] > 0.75)
).astype(int)

df["Segment_StickyButLowDeposit"] = (
    (df["Product_Stickiness"] > df["Product_Stickiness"].quantile(0.75)) & (df["Monthly_Deposits"] < df["Monthly_Deposits"].quantile(0.25))
).astype(int)

df["Segment_ZeroEngagement_OldTenure"] = (
    (df["Engagement_Momentum"] < 0.1) & (df["Account_Tenure"] > 5)
).astype(int)

df["Segment_DepositDependent_IncomeDrop"] = (
    (df["Deposit_Dependency"] > 0.8) & (df["Reported_Income_Increase"] == 0)
).astype(int)

df["Segment_MultipleProducts_LowBalance"] = (
    (df["Products_with_Bank"] > 3) & (df["Monthly_Avg_Balance"] < df["Monthly_Avg_Balance"].quantile(0.3))
).astype(int)

df["Segment_YoungAggressiveSaver"] = (
    (df["Age"] < 30) & (df["Monthly_Deposits"] > df["Monthly_Deposits"].quantile(0.75)) &
    (df["Deposit_Dependency"] > 0.8)
).astype(int)

df["Segment_ResidentHighOverdraft"] = (
    (df["Overdraft_Usage"] > 0.6) & (df["Customer_Type"].str.contains("Resident|National", case=False))
).astype(int)

df["Segment_OlderLowRiskLowEngagement"] = (
    (df["Age"] > 50) & (df["Risk_Profile_Score"] < df["Risk_Profile_Score"].quantile(0.25)) &
    (df["Engagement_Momentum"] < 0.3)
).astype(int)

df["Segment_YoungLowIncome_LowBalance"] = (
    (df["Age"] < 30) & (df["Income_Group"].isin(["Low"])) &
    (df["Monthly_Avg_Balance"] < df["Monthly_Avg_Balance"].quantile(0.3))
).astype(int)

df["Segment_LongTenure_DropInEngagement"] = (
    (df["Account_Tenure"] > 5) & (df["Engagement_Momentum"] < 0.2)
).astype(int)

df["Segment_StickyUser_LowLoyalty"] = (
    (df["Product_Stickiness"] > df["Product_Stickiness"].quantile(0.8)) & (df["Loyalty_Score"] < 0.4)
).astype(int)

df["Segment_BalanceFluctuation_Risk"] = (
    ((df["Monthly_Deposits"] - df["Monthly_Withdrawals"]).abs() > df["Monthly_Deposits"].std()) &
    (df["Risk_Profile_Score"] > df["Risk_Profile_Score"].quantile(0.75))
).astype(int)

df["Segment_SuspiciouslyInactive"] = (
    (df["Account_Activity"] < 0.1) & (df["Monthly_Withdrawals"] < 1) &
    (df["Monthly_Deposits"] < 1)
).astype(int)

# Save enriched dataset
df.to_csv("../data/features_with_segments.csv", index=False)


In [5]:
# 📌 Step X: Add Proxy Churn Features (Behavior-based Flags)

import pandas as pd
import numpy as np

# ✅ Load dataset
df = pd.read_csv("../data/features_with_segments.csv")

# ✅ 1. Recent Balance Drop (Simulate sudden cash withdrawal behavior)
df["Proxy_Recent_Balance_Drop"] = ((df["Monthly_Avg_Balance"] < df["Monthly_Deposits"] * 0.5) & (df["Monthly_Withdrawals"] > df["Monthly_Deposits"])).astype(int)

# ✅ 2. Sudden Drop in Engagement (Low activity but long tenure → possible churn)
df["Proxy_Engagement_Drop"] = ((df["Account_Tenure"] > 5) & (df["Account_Activity"] < 10)).astype(int)

# ✅ 3. Dormant High Value (High balance but low product use or activity)
df["Proxy_Dormant_High_Value"] = ((df["Monthly_Avg_Balance"] > 80000) & (df["Products_with_Bank"] <= 1) & (df["Account_Activity"] < 15)).astype(int)

# ✅ 4. Product Abandoner (Multiple products, no activity)
df["Proxy_Product_Abandoner"] = ((df["Products_with_Bank"] >= 3) & (df["Account_Activity"] < 5)).astype(int)

# ✅ 5. High Overdraft Stress (High loan EMI and overdraft usage)
df["Proxy_Overdraft_Stress"] = ((df["Loan_EMI"] > 10000) & (df["Overdraft_Usage"] > 0.7)).astype(int)

# ✅ 6. Loyalty Mismatch (High loyalty score but high withdrawals)
df["Proxy_Loyalty_Mismatch"] = ((df["Loyalty_Score"] > 0.8) & (df["Monthly_Withdrawals"] > 60000)).astype(int)

# ✅ 7. Income Spike but Low Activity (Sign of disengagement)
df["Proxy_Unresponsive_To_Income_Spike"] = ((df["Reported_Income_Increase"] == 1) & (df["Account_Activity"] < 10)).astype(int)

# ✅ 8. Risk But No Insurance (High risk profile, no insurance)
df["Proxy_Uninsured_High_Risk"] = ((df["Risk_Profile_Score"] > 0.7) & (df["Has_External_Insurance"] == 0)).astype(int)

# ✅ 9. Silent Dissatisfaction (Reported dissatisfaction + low complaints)
df["Proxy_Silent_Dissatisfaction"] = ((df["Reported_Dissatisfaction"] == 1) & (df["Unresolved_Complaints"] == 0)).astype(int)

# ✅ 10. Fluctuating Balance Pattern (Deposit-Withdraw instability)
df["Proxy_Balance_Instability"] = ((df["Monthly_Withdrawals"] - df["Monthly_Deposits"]).abs() > 0.6 * df["Monthly_Deposits"]).astype(int)

# ✅ 11. New Customer Risk (Low tenure and high withdrawal)
df["Proxy_New_Customer_Risk"] = ((df["Account_Tenure"] < 3) & (df["Monthly_Withdrawals"] > 50000)).astype(int)

# ✅ 12. High Value External Movement (High withdrawal + no income report)
df["Proxy_External_Money_Movement"] = ((df["Monthly_Withdrawals"] > 80000) & (df["Reported_Income_Increase"] == 0)).astype(int)

# ✅ 13. Inquired But No New Product (Curious but inactive)
df["Proxy_Inquired_No_Adoption"] = ((df["Inquired_New_Product"] == 1) & (df["Products_with_Bank"] <= 1)).astype(int)

# ✅ 14. Tenure-Withdrawal Conflict (Old customer suddenly withdrawing high amounts)
df["Proxy_Tenure_Withdrawal_Conflict"] = ((df["Account_Tenure"] > 10) & (df["Monthly_Withdrawals"] > 60000)).astype(int)

# ✅ 15. Churner Lookalike (Matches behavior of multiple churn personas)
df["Proxy_Churner_Lookalike"] = ((df["Segment_YoungAggressiveSaver"] == 1) | 
                                 (df["Segment_StickyUser_LowLoyalty"] == 1) |
                                 (df["Proxy_Overdraft_Stress"] == 1) |
                                 (df["Proxy_Engagement_Drop"] == 1)).astype(int)
# ✅ Proxy Flag 16: High Value but High Complexity
df["Proxy_HighValue_Complex"] = (
    (df["Monthly_Avg_Balance"] > 100000) &
    (df["Products_with_Bank"] >= 4) &
    (df["Loan_EMI"] > 15000)
).astype(int)

# ✅ Proxy Flag 17: Silent Drop-Off Pattern
df["Proxy_Silent_Disengage"] = (
    (df["Reported_Dissatisfaction"] == 0) &
    (df["Account_Activity"] < 10) &
    (df["Monthly_Deposits"] < 10000)
).astype(int)

# ✅ Proxy Flag 18: Insurance and Loan Confusion
df["Proxy_Confused_Loan_Insurance"] = (
    (df["Has_External_Insurance"] == 1) &
    (df["Other_Bank_Loans"] == 1) &
    (df["Loan_EMI"] > 12000)
).astype(int)

# ✅ Proxy Flag 19: Unexpected Overdraft Spike
df["Proxy_Overdraft_Spike"] = (
    (df["Overdraft_Usage"] > 0.95) &
    (df["Monthly_Avg_Balance"] < 15000)
).astype(int)

# ✅ Proxy Flag 20: Resigned Saver
df["Proxy_Resigned_Saver"] = (
    (df["Loyalty_Score"] < 0.3) &
    (df["Monthly_Deposits"] > 50000) &
    (df["Reported_Income_Increase"] == 0)
).astype(int)

# ✅ Proxy Flag 21: Product Explorer
df["Proxy_Product_Explorer"] = (
    (df["Inquired_New_Product"] == 1) &
    (df["Products_with_Bank"] >= 2) &
    (df["Monthly_Deposits"] < 15000)
).astype(int)

# ✅ Proxy Flag 22: Financial Conflict Persona
df["Proxy_Conflicted_Financial_State"] = (
    (df["Reported_Income_Increase"] == 1) &
    (df["Loan_EMI"] > 10000) &
    (df["Monthly_Withdrawals"] > 50000)
).astype(int)

# ✅ Proxy Flag 23: Prolonged Complaint Ignored
df["Proxy_Persistent_Complaint_Ignored"] = (
    (df["Unresolved_Complaints"] > 2) &
    (df["Reported_Dissatisfaction"] == 1)
).astype(int)

# ✅ Proxy Flag 24: Overloaded New Customer
df["Proxy_NewCustomer_Overload"] = (
    (df["Account_Tenure"] <= 3) &
    (df["Products_with_Bank"] > 3)
).astype(int)

# ✅ Proxy Flag 25: Underutilized Loyalist
df["Proxy_Loyal_UnderUtilized"] = (
    (df["Loyalty_Score"] > 0.9) &
    (df["Account_Activity"] < 5)
).astype(int)

# ✅ Save updated file
df.to_csv("../data/features_with_proxies.csv", index=False)
print("✅ Proxy churn features added and saved to features_with_proxies.csv")


✅ Proxy churn features added and saved to features_with_proxies.csv


In [6]:
# 📌 Add Composite Behavioral Indicators After Proxy Flags

import pandas as pd
import numpy as np

# ✅ Load latest dataset (after proxy flags)
df = pd.read_csv("../data/features_with_proxies.csv")

# ✅ Composite Behavioral Indicators
df["Engagement_Drop_Score"] = (
    (df["Loyalty_Score"] < 3).astype(int) * 0.4 +
    (df["Account_Activity"] < 20).astype(int) * 0.3 +
    (df["Monthly_Deposits"] < 10000).astype(int) * 0.3
)

df["RiskAware_Saver_Score"] = (
    df["Credit_Score"].replace({"Low": 1, "Medium": 2, "High": 3}) * 0.3 +
    df["Monthly_Avg_Balance"] / (df["Monthly_Deposits"] + 1) * 0.7
)

df["Overdraft_Dependency_Score"] = df["Overdraft_Usage"] * 0.5 + (df["Account_Activity"] < 15).astype(int) * 0.5

df["Frustration_Score"] = df["Unresolved_Complaints"] * 0.6 + df["Reported_Dissatisfaction"] * 0.4

df["Exit_Intent_Score"] = df["Inquired_New_Product"] * 0.4 + (df["Loyalty_Score"] < 3).astype(int) * 0.4 + df["Has_External_Insurance"] * 0.2

df["Financial_Pressure_Score"] = df["Loan_EMI"] * 0.5 + df["Credit_Score"].replace({"Low": 3, "Medium": 2, "High": 1}) * 0.3 + df["Other_Bank_Loans"] * 0.2

df["Passive_Relationship_Score"] = (df["Account_Tenure"] > 10).astype(int) * 0.5 + (df["Account_Activity"] < 10).astype(int) * 0.5

df["Confidence_Drop_Score"] = df["Reported_Dissatisfaction"] * 0.5 + (df["Loyalty_Score"] < 2).astype(int) * 0.3 + df["Issue_Reported"] * 0.2

df["Stickiness_Score"] = df["Products_with_Bank"] * 0.4 + df["Account_Tenure"] * 0.4 + (df["Loyalty_Score"] > 3).astype(int) * 0.2

df["Diminished_Engagement_Flag"] = ((df["Account_Activity"] < 10) & (df["Monthly_Deposits"] < 5000)).astype(int)

df["Silent_Flight_Risk"] = ((df["Issue_Reported"] == 0).astype(int) * 0.3 +
                            (df["Loyalty_Score"] < 2).astype(int) * 0.3 +
                            (df["Account_Activity"] < 5).astype(int) * 0.4)

df["False_Loyalty_Flag"] = ((df["Loyalty_Score"] > 4).astype(int) & (df["Unresolved_Complaints"] > 2)).astype(int)

df["Product_Confusion_Score"] = df["Products_with_Bank"] * 0.5 + df["Exit_Intent_Score"] * 0.5

df["Sudden_Inactivity_Flag"] = ((df["Account_Activity"] < 5) & (df["Monthly_Avg_Balance"] > 50000)).astype(int)

df["HighValue_Disengaged_Flag"] = ((df["Monthly_Avg_Balance"] > 100000) & (df["Account_Activity"] < 8)).astype(int)

df["RedAlert_Score"] = df["Frustration_Score"] * 0.5 + df["Exit_Intent_Score"] * 0.5

df["Undervalued_Customer_Score"] = ((df["Monthly_Avg_Balance"] > 80000).astype(int) * 0.4 +
                                    (df["Loyalty_Score"] < 2).astype(int) * 0.3 +
                                    df["Has_External_Insurance"] * 0.3)

df["Tenure_Risk_Mismatch_Flag"] = ((df["Account_Tenure"] > 10) &
                                   (df["Credit_Score"].replace({"Low": 1, "Medium": 0, "High": 0}) == 1)).astype(int)

df["Lost_Potential_Flag"] = ((df["Products_with_Bank"] >= 3) & (df["Loyalty_Score"] <= 2)).astype(int)

df["Churn_Heat_Index"] = (
    df["RedAlert_Score"] +
    df["Engagement_Drop_Score"] +
    df["Exit_Intent_Score"] +
    df["Financial_Pressure_Score"]
)

# ✅ Save enriched dataset
df.to_csv("../data/features_with_composites.csv", index=False)
print("✅ Composite behavioral indicators added and saved to features_with_composites.csv")


C:\Users\pullu\AppData\Local\Temp\ipykernel_50668\3702761245.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Credit_Score"].replace({"Low": 1, "Medium": 2, "High": 3}) * 0.3 +
C:\Users\pullu\AppData\Local\Temp\ipykernel_50668\3702761245.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Financial_Pressure_Score"] = df["Loan_EMI"] * 0.5 + df["Credit_Score"].replace({"Low": 3, "Medium": 2, "High": 1}) * 0.3 + df["Other_Bank_Loans"] * 0.2
C:\Users\pullu\AppData\Local\Temp\ipykernel_50668\3702761245.py:56: FutureWarning

✅ Composite behavioral indicators added and saved to features_with_composites.csv
